In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):    
    print(filenames)
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import pandas as pd
identification_csv = pd.read_csv("../input/dm2020-hw2-nthu/data_identification.csv")
emotion_csv = pd.read_csv("../input/dm2020-hw2-nthu/emotion.csv")

values = []
with open('../input/dm2020-hw2-nthu/tweets_DM.json', 'r') as tweet_data:
    for line in tweet_data.readlines():
        k = json.loads(line)
        values.append(k['_source']['tweet'])  ## 轉成json陣列 以方便可以直接轉成dataFrame
    

In [ ]:
raw_data = pd.DataFrame(values) 
print(raw_data)
raw_data = raw_data.sort_values(by=['tweet_id']) ## 依照tweet_id 做排序
identification_csv = identification_csv.sort_values(by=['tweet_id'])
ident_dict = identification_csv.set_index('tweet_id')['identification'].to_dict()## 將dataframe轉成dict 為了取出value的陣列
raw_data['identification'] = ident_dict.values() ## 直接加入陣列在新的column
raw_data

In [ ]:
## 依照'identification'的label做篩選 找出trainingData 和 testData
df_train = raw_data.loc[lambda raw_data: raw_data['identification'] == 'train',:]
df_test = raw_data.loc[lambda raw_data: raw_data['identification'] == 'test',:]

## 把emotion在df_train中加入新的column
df_train = df_train.sort_values(by=['tweet_id'])
emotion_csv = emotion_csv.sort_values(by=['tweet_id'])
emotion_dict = emotion_csv.set_index('tweet_id')['emotion'].to_dict()
df_train['emotion'] = emotion_dict.values()
print(df_train.shape)
df_train

In [ ]:
## save as pickle file
df_train.to_pickle("/kaggle/working/train_df.pkl") 
df_test.to_pickle("/kaggle/working/test_df.pkl") 


In [ ]:
import pandas as pd
# load a pickle file
train_df = pd.read_pickle("../input/tt-pickle-file/train_df.pkl")[:-5000]
test_df =  pd.read_pickle("../input/tt-pickle-file/train_df.pkl")[-5000:]
final_test_df = pd.read_pickle("../input/tt-pickle-file/test_df.pkl")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import nltk
# Initialize tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize,stop_words='english', min_df=100)

# Create tfidf train and test variables
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['text']) 
y_train = train_df['emotion']
X_test_tfidf = tfidf_vectorizer.transform(test_df['text'])
y_test = test_df['emotion']

# Create a LinearSVC model
tfidf_svc = LinearSVC()

# Train model
tfidf_svc.fit(X_train_tfidf, y_train)

# Run predict on tfidf test data to get predictions
tfidf_svc_pred = tfidf_svc.predict(X_test_tfidf)

# Calculate accuracy using the metrics module
tfidf_svc_score = tfidf_svc.score(X_test_tfidf, y_test)

print("LinearSVC Score:   %0.3f" % tfidf_svc_score)


In [ ]:
final_X_test_tfidf = tfidf_vectorizer.transform(final_test_df['text'])
final_tfidf_svc_pred = tfidf_svc.predict(final_X_test_tfidf)

final_test_df['emotion'] = final_tfidf_svc_pred
csvData = final_test_df[['tweet_id','emotion']].to_csv('/kaggle/working/predict_01.csv',header=['id','emotion'],index=False) 